In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


### Importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow as tf
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/merged_report.csv')

In [ ]:
df

,Reports,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,"""Nasogastric tube ends in the upper stomach bu...",1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,"""Cardiomediastinal silhouette is borderline en...",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
2,"""Compared to chest radiographs ___ through ___...",NaN,NaN,0.0,NaN,NaN,1.0,1.0,-1.0,-1.0,NaN,1.0,NaN,NaN,1.0
3,"""AP chest compared to ___: Nasogastric tube e...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0
4,"""PA and lateral views of the chest provided. ...",NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227819,"""ET tube in standard placement. Left jugular ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227820,"""Postoperative widening the cardiac silhouette...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227821,"""Pulmonary and mediastinal vascular congestion...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227822,"""In comparison to the chest radiograph obtaine...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#replacing NaN in the dataframe with 0s
df = df.fillna(0)


In [ ]:
# Seperating out the labels and the reports for training and testing
reports = df.drop(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices'], axis=1)

In [ ]:
labels = df.drop('Reports', axis = 1)

In [ ]:
reports

,Reports
0,"""Nasogastric tube ends in the upper stomach bu..."
1,"""Cardiomediastinal silhouette is borderline en..."
2,"""Compared to chest radiographs ___ through ___..."
3,"""AP chest compared to ___: Nasogastric tube e..."
4,"""PA and lateral views of the chest provided. ..."
...,...
227819,"""ET tube in standard placement. Left jugular ..."
227820,"""Postoperative widening the cardiac silhouette..."
227821,"""Pulmonary and mediastinal vascular congestion..."
227822,"""In comparison to the chest radiograph obtaine..."


In [ ]:
labels

,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Replacing all the negative values in the labels with 0
labels[labels<0]=0

In [ ]:
labels

,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
227822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# converting dataframe to numpy array
label_array = np.array(labels)

In [ ]:
label_array

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Defining the auxiliary functions to be used in text preprocessing

In [ ]:

def remove_punctuation(text):
  res = re.sub(r'[^\w\s]', '', text)
  return res

def remove_underscores(text):
  res = re.sub(r'_', '', text)
  return res

def remove_numbers(text):
  res = re.sub(r'[0-9]', '', text)
  return res

def stemmer_fun(text):
  text = text.split()
  stemmer = SnowballStemmer('english')
  stemmed_words = [stemmer.stem(word) for word in text]
  final_text = text = " ".join(stemmed_words)
  return final_text

### Defining the text preprocessing function.

In [ ]:
def text_preprocessing(text):
  text = text.str.lower()
  text = text.apply(lambda text: remove_punctuation(text))
  text = text.apply(lambda text: remove_numbers(text))
  text = text.apply(lambda text: remove_underscores(text))
  final_text = text.apply(lambda text: stemmer_fun(text))
  return final_text

In [ ]:
filtered_reports = text_preprocessing(reports['Reports'])

In [ ]:
report_array = np.array(filtered_reports)

### Using the tensorflow TextVectorization for encoding the text.

In [ ]:
encoder = tf.keras.layers.TextVectorization()
encoder.adapt(report_array)
encoded_text = encoder(report_array)
vocab = encoder.get_vocabulary()

In [ ]:
length_vocab = len(vocab)

In [ ]:
encoded_array  = np.array(encoded_text)

In [ ]:
encoded_array

array([[183,  26, 152, ...,   0,   0,   0],
       [ 43,  25,   3, ...,   0,   0,   0],
       [ 46,  16,  22, ...,   0,   0,   0],
       ...,
       [ 20,   6,  36, ...,   0,   0,   0],
       [  8,  65,  16, ...,   0,   0,   0],
       [ 55,  46,  16, ...,   0,   0,   0]])

In [ ]:
x_tv, x_test, y_tv, y_test = train_test_split(encoded_array, label_array, test_size=0.2, random_state=42)

In [ ]:
len(x_test)

45565

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_tv, y_tv, test_size = 0.2, random_state = 42)

In [ ]:
len(x_train)

145807

In [ ]:
len(y_train)

145807

### Creating a DataGenerator to create test, train and validation data to be feeded into the model.

In [ ]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, text, labels, batch_size=32,
                 n_classes=15, shuffle=True, validation=False, test = False, train = True):
        'Initialization'
        self.text = text
        encoder = tf.keras.layers.TextVectorization()
        encoder.adapt(self.text)
        self.encoded_text = encoder(self.text)
        self.labels = labels
        self.batch_size = batch_size
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.validation = validation
        self.test = test
        self.train = train
        x_tv, x_test, y_tv, y_test = train_test_split(self.encoded_text.numpy(), self.labels, test_size=0.2, random_state=42)
        x_train, x_valid, y_train, y_valid = train_test_split(x_tv, y_tv, test_size = 0.2, random_state = 42)
        if(self.validation):
          self.final_text = x_valid
          self.labels = y_valid
        if(self.test):
          self.final_text = x_test
          self.labels = y_test
        if(self.train):
          self.final_text = x_train
          self.labels = y_train
        self.on_epoch_end()
        

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.final_text) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
       # list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.labels))
        if self.shuffle == True:
          np.random.shuffle(self.indexes)

    def __data_generation(self, listed_index):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.final_text.shape[1]))
        X = X.astype(int)
        y = np.empty((self.batch_size, self.n_classes), dtype=int)
       
        # Generate data
        i = 0
        for j in listed_index:
            # Store sample
            X[i,] = self.final_text[j]

            # Store class
            y[i] = self.labels[j]
            i = i+1

        return X, y

In [ ]:
train_generator = DataGenerator(report_array, label_array, 32, 14, True, False, False, True)

In [ ]:
valid_generator = DataGenerator(report_array, label_array, 32, 14, True, True, False, False)

In [ ]:
test_generator = DataGenerator(report_array, label_array, 32, 14, True, False, True, False)

### Creating the first model with LSTM and Fully connected layers.

In [ ]:
model = tf.keras.Sequential()
# Configuring the parameters
model.add(tf.keras.layers.Embedding(10000, output_dim=1000, input_length=402))
model.add(tf.keras.layers.LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.LSTM(64, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
# Adding a dense output layer with sigmoid activation
model.add(tf.keras.layers.Dense(14,  activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 402, 1000)         10000000  
                                                                 
 lstm (LSTM)                 (None, 402, 128)          578048    
                                                                 
 dropout (Dropout)           (None, 402, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 402, 64)           49408     
                                                                 
 dropout_1 (Dropout)         (None, 402, 64)           0         
                                                                 
 flatten (Flatten)           (None, 25728)             0         
                                                                 
 dense (Dense)               (None, 14)                3

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    use_multiprocessing=True,
                    workers=6,  epochs = 10)

Epoch 1/10


<ipython-input-38-658eaab371a8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


4556/4556 [==============================] - 252s 53ms/step - loss: 0.0794 - val_loss: 0.0396
Epoch 2/10
4556/4556 [==============================] - 210s 46ms/step - loss: 0.0386 - val_loss: 0.0343
Epoch 3/10
4556/4556 [==============================] - 208s 45ms/step - loss: 0.0310 - val_loss: 0.0321
Epoch 4/10
4556/4556 [==============================] - 207s 45ms/step - loss: 0.0265 - val_loss: 0.0313
Epoch 5/10
4556/4556 [==============================] - 211s 46ms/step - loss: 0.0234 - val_loss: 0.0316
Epoch 6/10
4556/4556 [==============================] - 208s 46ms/step - loss: 0.0214 - val_loss: 0.0299
Epoch 7/10
4556/4556 [==============================] - 208s 46ms/step - loss: 0.0195 - val_loss: 0.0313
Epoch 8/10
4556/4556 [==============================] - 208s 45ms/step - loss: 0.0184 - val_loss: 0.0317
Epoch 9/10
4556/4556 [==============================] - 207s 45ms/step - loss: 0.0174 - val_loss: 0.0316
Epoch 10/10
4556/4556 [==============================] - 207s 45ms

In [ ]:
y_predict = model.predict(x_test)

1424/1424 [==============================] - 21s 14ms/step


In [ ]:
len(y_predict)

45565

In [ ]:
y_predict[0]

array([4.4983015e-12, 5.5344263e-04, 9.9987495e-01, 3.9015966e-05,
       1.7434922e-04, 3.0264890e-04, 1.1245811e-04, 2.8058657e-04,
       9.9475092e-01, 2.2056283e-04, 9.9889010e-01, 2.6660167e-05,
       7.3806819e-05, 9.9942940e-01], dtype=float32)

In [ ]:
model.evaluate(x_test, y_test)

1424/1424 [==============================] - 23s 16ms/step - loss: 0.0312


0.0311922337859869

### Evaluating the accuracy metrics for first model for different threshold values of the sigmoid function.

In [ ]:
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=y_predict.copy()
    pred[pred>=val]=1
    pred[pred<val]=0
    print(classification_report(y_test, pred))
  

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8228
           1       0.90      0.97      0.94      5819
           2       0.96      0.99      0.98     14146
           3       0.93      0.98      0.96      1647
           4       0.97      1.00      0.98     13753
           5       0.95      0.98      0.97      5681
           6       0.91      0.97      0.94      2523
           7       0.81      0.96      0.88      2993
           8       0.97      0.99      0.98     11171
           9       0.92      0.94      0.93      2443
          10       0.97      0.99      0.98     12382
          11       0.90      0.99      0.94       713
          12       0.96      0.98      0.97      1488
          13       0.98      1.00      0.99     17415

   micro avg       0.95      0.99      0.97    100402
   macro avg       0.93      0.98      0.95    100402
weighted avg       0.95      0.99      0.97    100402
 samples avg       0.91   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      8228
           1       0.94      0.96      0.95      5819
           2       0.97      0.99      0.98     14146
           3       0.95      0.97      0.96      1647
           4       0.98      0.99      0.98     13753
           5       0.97      0.98      0.97      5681
           6       0.93      0.96      0.95      2523
           7       0.88      0.93      0.91      2993
           8       0.97      0.99      0.98     11171
           9       0.95      0.93      0.94      2443
          10       0.98      0.98      0.98     12382
          11       0.92      0.98      0.95       713
          12       0.97      0.97      0.97      1488
          13       0.99      0.99      0.99     17415

   micro avg       0.97      0.98      0.97    100402
   macro avg       0.95      0.97      0.96    100402
weighted avg       0.97      0.98      0.97    100402
 samples avg       0.92   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

           0       0.95      0.98      0.97      8228
           1       0.95      0.94      0.95      5819
           2       0.98      0.99      0.98     14146
           3       0.95      0.96      0.96      1647
           4       0.98      0.99      0.99     13753
           5       0.97      0.97      0.97      5681
           6       0.95      0.95      0.95      2523
           7       0.91      0.92      0.91      2993
           8       0.98      0.99      0.98     11171
           9       0.96      0.92      0.94      2443
          10       0.98      0.98      0.98     12382
          11       0.93      0.98      0.96       713
          12       0.97      0.97      0.97      1488
          13       0.99      0.99      0.99     17415

   micro avg       0.97      0.98      0.98    100402
   macro avg       0.96      0.97      0.96    100402
weighted avg       0.97      0.98      0.98    100402
 samples avg       0.92   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      8228
           1       0.97      0.93      0.95      5819
           2       0.98      0.98      0.98     14146
           3       0.96      0.95      0.96      1647
           4       0.98      0.99      0.99     13753
           5       0.98      0.96      0.97      5681
           6       0.96      0.95      0.95      2523
           7       0.93      0.90      0.91      2993
           8       0.98      0.98      0.98     11171
           9       0.97      0.91      0.94      2443
          10       0.99      0.97      0.98     12382
          11       0.94      0.97      0.96       713
          12       0.98      0.96      0.97      1488
          13       0.99      0.99      0.99     17415

   micro avg       0.98      0.97      0.98    100402
   macro avg       0.97      0.96      0.96    100402
weighted avg       0.98      0.97      0.98    100402
 samples avg       0.92   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Creating the second model with functional API of tensorflow as it used parallel layers.

In [ ]:
class model2(tf.keras.Model):
  def __init__(self, num_classes=1000):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(length_vocab, output_dim=1000, input_length=402)
        self.lstm1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        self.lstm2 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.linear1 = tf.keras.layers.Dense(256, activation = 'relu')
        self.linear2 = tf.keras.layers.Dense(256, activation = 'relu')
        self.linear3 = tf.keras.layers.Dense(128, activation = 'relu')
        self.final = tf.keras.layers.Dense(14, activation = 'sigmoid')
  def call(self, input):
        h_embeddings = self.embedding(input)
        h_lstm1 = self.lstm1(h_embeddings)
        h_lstm1 = self.dropout1(h_lstm1)
        h_lstm2 = self.lstm2(h_lstm1)
        h_lstm2 = self.dropout2(h_lstm2)
        # global average pooling
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(h_lstm2)
        # global max pooling
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(h_lstm2)
        h_conc = tf.keras.layers.Concatenate(axis=1)([avg_pool, max_pool])
        h_conc_linear1 = self.linear1(h_conc)
        h_conc_linear2 = self.linear2(h_conc)
        h_conc_linear3 = tf.keras.layers.Concatenate(axis=1)([h_conc_linear1, h_conc_linear2])
        h_conc_linear3 = self.linear3(h_conc_linear3)
        final_output = self.final(h_conc_linear3)
                                                    
        return final_output


In [ ]:
new_model = model2()

In [ ]:
new_model.build((None, 402))

In [ ]:
new_model.summary()

Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  11392000  
                                                                 
 lstm_2 (LSTM)               multiple                  578048    
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 lstm_3 (LSTM)               multiple                  131584    
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  65792     
                                                                 
 dense_2 (Dense)             multiple                  65792

In [ ]:
new_model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history_1  = new_model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    use_multiprocessing=True,
                    workers=6,  epochs = 10)

Epoch 1/10


<ipython-input-49-6bf51e37b42c>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_1  = new_model.fit_generator(generator=train_generator,


4556/4556 [==============================] - 270s 58ms/step - loss: 0.0834 - val_loss: 0.0705
Epoch 2/10
4556/4556 [==============================] - 222s 49ms/step - loss: 0.0327 - val_loss: 0.0719
Epoch 3/10
4556/4556 [==============================] - 221s 48ms/step - loss: 0.0269 - val_loss: 0.0715
Epoch 4/10
4556/4556 [==============================] - 221s 48ms/step - loss: 0.0237 - val_loss: 0.0694
Epoch 5/10
4556/4556 [==============================] - 220s 48ms/step - loss: 0.0216 - val_loss: 0.0596
Epoch 6/10
4556/4556 [==============================] - 221s 48ms/step - loss: 0.0203 - val_loss: 0.0519
Epoch 7/10
4556/4556 [==============================] - 220s 48ms/step - loss: 0.0192 - val_loss: 0.0650
Epoch 8/10
4556/4556 [==============================] - 219s 48ms/step - loss: 0.0182 - val_loss: 0.0564
Epoch 9/10
4556/4556 [==============================] - 220s 48ms/step - loss: 0.0177 - val_loss: 0.0513
Epoch 10/10
4556/4556 [==============================] - 220s 48ms

In [ ]:
y1_predict = new_model.predict(x_test)

1424/1424 [==============================] - 22s 15ms/step


### Evaluating the accuracy metrics for the second model for different threshold values of the sigmoid function.

In [ ]:
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=y1_predict.copy()
    pred[pred>=val]=1
    pred[pred<val]=0
    print(classification_report(y_test, pred))

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.62      1.00      0.76      8228
           1       0.91      0.97      0.94      5819
           2       0.97      0.99      0.98     14146
           3       0.92      0.98      0.95      1647
           4       0.98      0.99      0.99     13753
           5       0.94      0.98      0.96      5681
           6       0.92      0.97      0.94      2523
           7       0.70      0.97      0.81      2993
           8       0.97      0.99      0.98     11171
           9       0.92      0.94      0.93      2443
          10       0.97      0.99      0.98     12382
          11       0.92      0.97      0.94       713
          12       0.96      0.98      0.97      1488
          13       0.98      1.00      0.99     17415

   micro avg       0.91      0.99      0.95    100402
   macro avg       0.90      0.98      0.94    100402
weighted avg       0.93      0.99      0.95    100402
 samples avg       0.88   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.72      1.00      0.84      8228
           1       0.95      0.96      0.95      5819
           2       0.98      0.99      0.98     14146
           3       0.95      0.97      0.96      1647
           4       0.98      0.99      0.99     13753
           5       0.97      0.97      0.97      5681
           6       0.96      0.94      0.95      2523
           7       0.84      0.95      0.89      2993
           8       0.98      0.99      0.98     11171
           9       0.95      0.93      0.94      2443
          10       0.98      0.98      0.98     12382
          11       0.95      0.94      0.95       713
          12       0.98      0.97      0.97      1488
          13       0.99      1.00      0.99     17415

   micro avg       0.94      0.98      0.96    100402
   macro avg       0.94      0.97      0.95    100402
weighted avg       0.95      0.98      0.96    100402
 samples avg       0.90   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.94      0.99      0.96      8228
           1       0.99      0.71      0.83      5819
           2       0.99      0.91      0.95     14146
           3       0.98      0.84      0.90      1647
           4       0.99      0.95      0.97     13753
           5       0.99      0.80      0.89      5681
           6       0.99      0.68      0.81      2523
           7       0.96      0.84      0.89      2993
           8       0.99      0.95      0.97     11171
           9       0.99      0.78      0.87      2443
          10       0.99      0.88      0.93     12382
          11       0.99      0.58      0.73       713
          12       1.00      0.78      0.88      1488
          13       0.99      0.97      0.98     17415

   micro avg       0.99      0.90      0.94    100402
   macro avg       0.98      0.83      0.90    100402
weighted avg       0.99      0.90      0.94    100402
 samples avg       0.89   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.98      0.97      8228
           1       0.99      0.51      0.68      5819
           2       0.99      0.81      0.89     14146
           3       0.98      0.72      0.83      1647
           4       0.99      0.90      0.94     13753
           5       0.99      0.66      0.79      5681
           6       1.00      0.52      0.68      2523
           7       0.97      0.76      0.85      2993
           8       0.99      0.91      0.95     11171
           9       0.99      0.64      0.78      2443
          10       1.00      0.75      0.86     12382
          11       1.00      0.46      0.63       713
          12       1.00      0.62      0.77      1488
          13       0.99      0.93      0.96     17415

   micro avg       0.99      0.82      0.89    100402
   macro avg       0.99      0.73      0.83    100402
weighted avg       0.99      0.82      0.89    100402
 samples avg       0.85   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
